# Problem 1: MLE for GARCH/TGARCH Parameter Estimation

## (a)

Given $$\sigma_t^2 = \omega + \alpha \cdot \epsilon_{t-1}^2 + \beta \cdot \sigma_{t-1}^2$$

We want to maximize

$$\log L = -\frac{1}{2} \sum_{t=1}^T \left( \log(2\pi) + \log(\sigma_t^2) + \frac{\epsilon_t^2}{\sigma_t^2} \right)$$

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import yfinance as yf

# Step 1: Fetch SPY data
spy_data = yf.download('SPY', start='1993-02-01', end='2023-10-20')

# Step 2: Calculate and rescale log returns
spy_data['log_return'] = 100 * np.log(spy_data['Adj Close'] / spy_data['Adj Close'].shift(1))  # Rescaling by 100
returns = spy_data['log_return'].dropna()

# Step 3: Define the GARCH(1,1) likelihood function
def garch_likelihood(params, data):
    alpha0, alpha1, beta1 = params
    if alpha1 + beta1 >= 1:  # Enforcing stationarity
        return np.inf
    n = len(data)
    sigma2 = np.full(n, np.var(data))  # More stable initialization
    log_likelihood = 0.0

    for t in range(1, n):
        sigma2[t] = max(alpha0 + alpha1 * data[t-1]**2 + beta1 * sigma2[t-1], 1e-8)
        log_likelihood += -0.5 * (np.log(2 * np.pi) + np.log(sigma2[t]) + data[t]**2 / sigma2[t])

    return -log_likelihood

# Step 4: Optimize the GARCH(1,1) parameters
# Adjust initial_params to reflect the rescaling
initial_params = [0.1, 0.05, 0.9]  # Adjusted initial guess for alpha0
result = minimize(garch_likelihood, initial_params, args=(returns.values,), method='SLSQP', bounds=[(0, None), (0, 1), (0, 1)], options={'maxiter': 10000, 'ftol': 1e-10})

# Step 5: Output the results
if result.success:
    fitted_params = result.x
    max_likelihood = -result.fun
    print("Estimated Parameters:", fitted_params)
    print("Maximum Log-Likelihood:", max_likelihood)
else:
    print("Optimization failed:", result.message)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
[*********************100%%**********************]  1 of 1 completed
Estimated Parameters: [0.01921922 0.10808238 0.87787505]
Maximum Log-Likelihood: -10551.148052375826


The SBC/BIC is computed as:

$$ \text{BIC} = k \ln(n) - 2 \ln(\hat{L}) $$

In [2]:
k = len(fitted_params)
n = len(returns)
bic = k * np.log(n) - 2 * max_likelihood
print("BIC:", bic)

BIC: 21129.156637040673


## (b)

In [3]:
from scipy.stats import gennorm

def garch_ged_likelihood(params, data):
    alpha0, alpha1, beta1, nu = params
    n = len(data)
    sigma2 = np.full(n, np.var(data))  # Stable initialization
    log_likelihood = 0.0

    for t in range(1, n):
        sigma2[t] = max(alpha0 + alpha1 * data[t-1]**2 + beta1 * sigma2[t-1], 1e-8)
        pdf_val = gennorm.pdf(data[t], nu, scale=np.sqrt(sigma2[t]))
        if pdf_val <= 0:
            return np.inf  # Penalize invalid PDF values
        log_likelihood += np.log(pdf_val)

    return -log_likelihood


alpha0_guess = 0.1 
alpha1_guess = 0.05 
beta1_guess = 0.9  
nu_guess = 2.0      

initial_params = [alpha0_guess, alpha1_guess, beta1_guess, nu_guess]

result = minimize(garch_ged_likelihood, initial_params, args=(returns.values,), method='SLSQP', bounds=[(0, None), (0, 1), (0, 1), (1, None)])

if result.success:
    fitted_params = result.x
    max_likelihood = -result.fun
    print("Estimated Parameters:", fitted_params)
    print("Maximum Log-Likelihood:", max_likelihood)
else:
    print("Optimization failed:", result.message)


/Users/Eric/opt/anaconda3/envs/dsm/lib/python3.11/site-packages/scipy/stats/_continuous_distns.py:10348: RuntimeWarning: overflow encountered in power
  return np.log(0.5*beta) - sc.gammaln(1.0/beta) - abs(x)**beta


Estimated Parameters: [0.01570085 0.11505633 0.88512925 1.33235826]
Maximum Log-Likelihood: -10374.577239663622


In [4]:
k = len(fitted_params)
n = len(returns)
bic = k * np.log(n) - 2 * max_likelihood
print("BIC:", bic)

BIC: 20784.968522379273


### Conclusion: 
Since the BIC of the GARCH-GED model is lower, it suggests that the improved fit (higher likelihood) compensates for the added complexity of the extra parameter.

## (c)

In [5]:
def tgarch_likelihood(params, data):
    alpha0, alpha1, gamma, beta1 = params
    # We'll handle the stationarity condition in the bounds
    n = len(data)
    sigma2 = np.full(n, np.var(data))
    log_likelihood = 0.0

    for t in range(1, n):
        indicator = 1 if data[t-1] < 0 else 0
        sigma2[t] = max(alpha0 + (alpha1 + gamma * indicator) * data[t-1]**2 + beta1 * sigma2[t-1], 1e-8)
        log_likelihood += -0.5 * (np.log(2 * np.pi) + np.log(sigma2[t]) + data[t]**2 / sigma2[t])

    return -log_likelihood

# Adjusted initial parameters and bounds
initial_params_tgarch = [0.1, 0.05, 0.05, 0.8]  # Adjust these as needed
bounds_tgarch = [(1e-6, None), (0, 0.5), (0, 0.5), (0, 0.99)]  # Looser bounds, ensuring stationarity

result_tgarch = minimize(tgarch_likelihood, initial_params_tgarch, args=(returns.values,), method='SLSQP', bounds=bounds_tgarch, options={'maxiter': 100000, 'ftol': 1e-6})

if result_tgarch.success:
    fitted_params_tgarch = result_tgarch.x
    max_likelihood_tgarch = -result_tgarch.fun
    print("TGARCH(1,1) Estimated Parameters:", fitted_params_tgarch)
    print("TGARCH(1,1) Maximum Log-Likelihood:", max_likelihood_tgarch)
else:
    print("TGARCH(1,1) Optimization failed:", result_tgarch.message)

k_tgarch = 4  
n = len(returns)  
bic_tgarch = -2 * max_likelihood_tgarch + k_tgarch * np.log(n)
print("BIC:", bic_tgarch)

TGARCH(1,1) Estimated Parameters: [0.02241267 0.00554826 0.17445458 0.89035083]
TGARCH(1,1) Maximum Log-Likelihood: -10404.512639651963
BIC: 20844.839322355954


### Conclusion: 
Since the BIC of the TGARCH(1,1) model is lower, it suggests that the improved fit (higher likelihood) compensates for the added complexity of the extra parameter.

# Probelm 2: Straddle Pricing with GARCH/TGARCH

One of the big trends in options this year is the dramatic rise of trading in zero-daysto-expiry (0DTE) options . These are options that expire within 24 hours. Options
that expire every day of the week are now available for popular securities like SPY and
QQQ. According to the CBOE, 44% of the S&P500 option volume is for options with
less than 24 hours to expiration, for a notional amount of over $500bn traded per day.
The figure below lists prices and implied volatilities for puts and calls on SPY options
as of the close on Friday, October 20, 2023. The middle column are the strikes, and
the data on the left side are for the call options, and the data on the right side are for
the put options. The first five rows are for options that expire the following Monday,
one trading day later.

Use the GARCH model in part (a) and the TGARCH model in part (c) above to
price the 421 straddle (a straddle is a combination of a put and call at the same strike
and expiration). You should price the straddle using Monte Carlo simulations over a
one-day horizon. Explain how you get the volatility forecast from Friday to Monday.
Notice that the market price of this straddle is 2.05 + 2.10 = 4.15. Do the models want
to buy or sell the straddle? The TGARCH model should place a higher value on these
options. Give a one sentence explanation why.

In [6]:
import numpy as np
import pandas as pd
import yfinance as yf

# Parameters for GARCH(1,1) and TGARCH
params_garch = [0.01921938, 0.10808369, 0.87787375]
params_tgarch = [0.02241294, 0.00554883, 0.17445635, 0.89034935]

# Fetch SPY data
spy_data = yf.download('SPY', start='1993-02-01', end='2023-10-20')
spy_data['log_return'] = 100 * np.log(spy_data['Adj Close'] / spy_data['Adj Close'].shift(1))  # Rescaling by 100
returns = spy_data['log_return'].dropna()

# Last known price and return
last_price = spy_data['Adj Close'].iloc[-1]
last_return = returns.iloc[-1]

# Number of simulations
num_simulations = 10000

# Adjusted functions to simulate log returns with checks for non-negative variance

# Function to simulate log returns using GARCH(1,1) model with non-negative variance check
def simulate_garch_fixed(params, last_return, num_simulations, initial_variance):
    alpha0, alpha1, beta1 = params
    sim_returns = np.zeros(num_simulations)
    variance = initial_variance
    
    for i in range(num_simulations):
        random_shock = np.random.normal(0, np.sqrt(max(variance, 1e-8)))
        variance = alpha0 + alpha1 * last_return**2 + beta1 * variance
        sim_returns[i] = random_shock * np.sqrt(max(variance, 1e-8))
        
    return sim_returns

# Function to simulate log returns using TGARCH model with non-negative variance check
def simulate_tgarch_fixed(params, last_return, num_simulations, initial_variance):
    alpha0, alpha1, gamma, beta1 = params
    sim_returns = np.zeros(num_simulations)
    variance = initial_variance
    
    for i in range(num_simulations):
        indicator = 1 if last_return < 0 else 0
        random_shock = np.random.normal(0, np.sqrt(max(variance, 1e-8)))
        variance = alpha0 + (alpha1 + gamma * indicator) * last_return**2 + beta1 * variance
        sim_returns[i] = random_shock * np.sqrt(max(variance, 1e-8))
        
    return sim_returns

# Assume an initial variance based on the long-term average variance of the returns
initial_variance = np.var(returns)

# Straddle strike price
strike_price = 421

# Simulate future log returns with adjusted functions
sim_returns_garch_fixed = simulate_garch_fixed(params_garch, last_return, num_simulations, initial_variance)
sim_returns_tgarch_fixed = simulate_tgarch_fixed(params_tgarch, last_return, num_simulations, initial_variance)

# Calculate future prices with adjusted simulations
future_prices_garch_fixed = last_price * np.exp(sim_returns_garch_fixed / 100)  # Rescaling back
future_prices_tgarch_fixed = last_price * np.exp(sim_returns_tgarch_fixed / 100)  # Rescaling back

# Calculate payouts and average price of the straddle for both models
straddle_price_garch_fixed = np.mean(np.abs(future_prices_garch_fixed - strike_price))
straddle_price_tgarch_fixed = np.mean(np.abs(future_prices_tgarch_fixed - strike_price))

straddle_price_garch_fixed, straddle_price_tgarch_fixed

[*********************100%%**********************]  1 of 1 completed


(5.584295767852295, 6.747386510593331)

Comparing these to the market price of the straddle, which is $\$4.15$, we can infer the following:

**GARCH Model**: The model's price of $\$5.62$ is higher than the market price. This suggests that, according to the GARCH model, the straddle is undervalued in the market, and it would be a buy signal.

**TGARCH Model**: With an even higher model price of $\$6.69$, the TGARCH model also indicates that the straddle is undervalued in the market, suggesting a buy.
The TGARCH model assigns a higher value to the straddle compared to the GARCH model. This is likely because the TGARCH model accounts for leverage effects, where negative returns increase future volatility more than positive returns. In volatile markets, this often results in higher option valuations under the TGARCH model, as options become more valuable with increased volatility expectations.

# Problem 3: Verify  OLS as a special case of GMM

In [7]:
from scipy.optimize import minimize
import numpy as np

# Set the true values for alpha and beta for the data simulation
alpha_true = 1
beta_true = 2
N = 100  # Number of data points
sigma = 1  # Standard deviation of the error terms

# Simulate the independent variable x and the error terms
np.random.seed(0)
x = np.random.rand(N) * 10  # Let's take x to be uniformly distributed between 0 and 10
errors = np.random.normal(0, sigma, N)

# Simulate the dependent variable y
y = alpha_true + beta_true * x + errors

# Define the moment conditions for GMM
def moment_conditions(params, x, y):
    alpha, beta = params
    residuals = y - (alpha + beta * x)
    m1 = residuals  # Moment condition based on the error term
    m2 = x * residuals  # Moment condition based on the error term times x
    return np.array([np.mean(m1), np.mean(m2)])

# Objective function for GMM is the squared sum of moment conditions
def gmm_objective(params, x, y):
    moments = moment_conditions(params, x, y)
    return np.sum(moments**2)

# Initial guesses for alpha and beta
initial_guess = [0, 0]

# Minimize the GMM objective to find the estimated alpha and beta
gmm_result = minimize(gmm_objective, initial_guess, args=(x, y))

# The GMM estimated parameters are in gmm_result.x
gmm_estimated_alpha, gmm_estimated_beta = gmm_result.x

# Print the estimated parameters
print(f'GMM Estimated alpha: {gmm_estimated_alpha}')
print(f'GMM Estimated beta: {gmm_estimated_beta}')


GMM Estimated alpha: 1.2221641996552628
GMM Estimated beta: 1.993691454046184


### Verification

In [8]:
import statsmodels.api as sm

# Add a constant term to x to include an intercept in the model
X = sm.add_constant(x)

# Create the OLS model and fit it to the data
model = sm.OLS(y, X)
results = model.fit()

# Retrieve and print the estimated parameters from the OLS model
ols_estimated_params = results.params
ols_estimated_params

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


array([1.22215108, 1.9936935 ])

# Problem 4: Estimate CKLS Short-Term Parameters

In the paper “An Empirical Comparison of Alternative Models of the Short-Term Interest Rate” by Chan, Karolyi, Longstaff, and Sanders (CKLS), they estimate parameters for eight popular models of short-term interest rates using GMM. In this exercise, you will replicate their study on one of those models.

(a) Estimate the four parameters of the unrestricted model.

(b) Estimate the two parameters in the Geometric Brownian Motion (GBM) model.

Just like with the other excercises, I would like you to estimate the parameters from
scratch, rather than using a package. I have attached one-month interest rate data
from WRDS (the start date of 1964-06-30 and the end date of 1989-12-29 was chosen
to match CKLS). I am also attaching the original CKLS paper. I was able to match
the parameters of Table III of their paper relatively closely, but not exactly. In order
to match the parameters, the data needs to be scaled correctly. First of all, you should
divide the interest rates by 100, since 5% is represented as 5.0 and not 0.05. You should
also include ∆t = 1/12 in the moment conditions. The first moment condition, for example, should be:

$$E[r_{t+1} − r_t − (α + βr_t)∆t] = 0$$

and the second moment condition should be:

$$E[ϵ_{t+1}^2 − σ^2 r_t^{2γ}∆t] = 0$$

Also, to make things easier, you can choose to do two-step GMM rather than iterated
GMM, and you can compute the weighting matrix assuming the data are uncorrelated
so you don’t have to compute the Newey-West adjustment if you would like. You may
have to vary the initial parameter guesses because sometimes the optimizer gets stuck
in a local maximum that is not the global maximum. 

## (a)

In [9]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Load the data
file_path = '/Users/Eric/Downloads/Tbill_1mo.csv' 
interest_rate_data = pd.read_csv(file_path)

# Prepare the data by scaling the interest rates
interest_rate_data['scaled_rate'] = interest_rate_data['ave_1'] / 100

# Calculate the time step for monthly data
delta_t = 1 / 12

# Get the interest rates from the data
interest_rates = interest_rate_data['scaled_rate'].values

# Define the moment conditions
def moment_conditions(params, rates, delta_t):
    alpha, beta, sigma_sq, gamma = params
    residuals_first = rates[1:] - rates[:-1] - (alpha + beta * rates[:-1]) * delta_t
    residuals_second = residuals_first * rates[:-1]
    residuals_third = (residuals_first**2 - sigma_sq * (rates[:-1]**(2 * gamma)) * delta_t)
    return np.column_stack((residuals_first, residuals_second, residuals_third))

# Objective function for GMM
def gmm_objective(params, rates, delta_t, weight_matrix):
    model_residuals = moment_conditions(params, rates, delta_t)
    g_bar = np.mean(model_residuals, axis=0)
    return g_bar @ weight_matrix @ g_bar

# Initial parameter guesses
initial_params = np.array([0, 0, 1.5, 2])

# Identity matrix for the first step
weight_identity = np.eye(3)

# First step optimization using the identity matrix for weighting
result_first_step = minimize(
    fun=gmm_objective,
    x0=initial_params,
    args=(interest_rates, delta_t, weight_identity),
    method='L-BFGS-B',
    bounds=[(None, None), (None, None), (0, None), (0, None)]
)

if not result_first_step.success:
    raise Exception("First step optimization failed.")

# Calculate the optimal weighting matrix using the residuals from the first step
model_residuals_first_step = moment_conditions(result_first_step.x, interest_rates, delta_t)
S_matrix = np.mean([np.outer(resid, resid) for resid in model_residuals_first_step], axis=0)
optimal_weight = np.linalg.inv(S_matrix)

# Second step optimization using the optimal weighting matrix
result_second_step = minimize(
    fun=gmm_objective,
    x0=result_first_step.x,
    args=(interest_rates, delta_t, optimal_weight),
    method='L-BFGS-B',
    bounds=[(None, None), (None, None), (0, None), (0, None)]
)

if result_second_step.success:
    estimated_params = result_second_step.x
    print(f"Estimated parameters (two-step GMM): {estimated_params}")
else:
    raise Exception("Second step optimization failed.")


Estimated parameters (two-step GMM): [ 0.04137383 -0.60433151  1.57585062  1.48397778]


The parameters for the unrestricted model have been estimated using the two-step GMM approach. The estimated parameters are as follows:
1. $ \alpha $: $0.04137383$
2. $ \beta $: $-0.60433151$
3. $ \sigma^2 $: $1.57585062$
4. $ \gamma $: $1.48397778$

## (b)

In [10]:
# Load the data
file_path = '/Users/Eric/Downloads/Tbill_1mo.csv' 
interest_rate_data = pd.read_csv(file_path)

# Prepare the data by scaling the interest rates
interest_rate_data['scaled_rate'] = interest_rate_data['ave_1'] / 100

# Calculate the time step for monthly data
delta_t = 1 / 12

# Get the interest rates from the data
interest_rates = interest_rate_data['scaled_rate'].values

# Define the moment conditions with fixed alpha = 0 and gamma = 1
def moment_conditions(params, rates, time_step):
    beta, sigma_sq = params
    residuals_first = rates[1:] - rates[:-1] - (beta * rates[:-1]) * time_step
    residuals_second = residuals_first * rates[:-1]
    residuals_third = (residuals_first**2 - sigma_sq * (rates[:-1]**2) * time_step)
    all_residuals = np.column_stack((residuals_first, residuals_second, residuals_third))
    return all_residuals

# Define the objective function for the GMM first step
def gmm_objective_step1(params, rates, time_step):
    residuals = moment_conditions(params, rates, time_step)
    weighting_matrix = np.eye(residuals.shape[1])
    criterion_value = residuals.mean(axis=0) @ weighting_matrix @ residuals.mean(axis=0)
    return criterion_value

# Initial parameter guesses for beta and sigma^2
initial_params = np.array([0.1, 0.1])

# First step optimization with fixed alpha and gamma
result_step1 = minimize(
    fun=gmm_objective_step1,
    x0=initial_params,
    args=(interest_rates, delta_t),
    method='L-BFGS-B',
    bounds=[(None, None), (0, None)]
)

# Calculate the optimal weighting matrix using the residuals from the first step
residuals_step1 = moment_conditions(result_step1.x, interest_rates, delta_t)
optimal_weighting_matrix = np.linalg.inv(np.cov(residuals_step1.T))

# Define the objective function for the GMM second step
def gmm_objective_step2(params, rates, time_step, W):
    residuals = moment_conditions(params, rates, time_step)
    criterion_value = residuals.mean(axis=0) @ W @ residuals.mean(axis=0)
    return criterion_value

# Second step optimization using the optimal weighting matrix
result_step2 = minimize(
    fun=gmm_objective_step2,
    x0=result_step1.x,
    args=(interest_rates, delta_t, optimal_weighting_matrix),
    method='L-BFGS-B',
    bounds=[(None, None), (0, None)]
)

# Check the result and print the parameters
if result_step2.success:
    estimated_params = result_step2.x
    print(f"Estimated parameters (two-step GMM): {estimated_params}")
else:
    raise Exception("Second step optimization failed: " + result_step2.message)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Estimated parameters (two-step GMM): [0.10564767 0.12328659]


The parameters for GBM model have been estimated using the two-step GMM approach. The estimated parameters are as follows:

1. $\beta$: $0.10564767$

2. $\sigma^2$: $0.12328659$